## MCMC

## Import

In [15]:
import numpy as np
import pandas as pd
import os
import emcee
import multiprocessing
import scipy.integrate as integrate
import math
import matplotlib.pyplot as plt
import corner
plt.rcParams.update({'font.size': 12})
from tqdm import tqdm
import scipy.linalg as la

## Constants

In [17]:
# Define file paths
base_directory = 'D:\\OneDrive\\Documents\\progetto_chiura\\mcmc\\save'
covariance_directory = 'D:\\OneDrive\\Documents\\progetto_chiura\\mcmc\\save\\covariance_matrix'

# Define constants
G = 4.30091e-6

# Define directories and variables
star_number = 100
anisotropy_type = 'iso'
profile_type = 'core'
space_type = 'cartesian'
print_matrix = 0  # Set this to choose which covariance matrix to print

nwalker = 50 #50
ndim = 3 #this dfines the number of model parameters that required to be constrained.
niter = 500 #500



integrand1 = lambda r: r**2 * (rho0 / (((r / rs)**gamma) * (1 + r / rs)**(3 - gamma)))

# Loaders

In [18]:
#Create mockdata
def load_data(file_path):
    data = pd.read_csv(file_path, delim_whitespace=True, names=['x', 'y', 'z', 'vx', 'vy', 'vz'])

    return data.to_numpy()  # Convert the DataFrame to a NumPy array, 
        
#Create cov matrixes
def load_covariance_matrix(star_index, anisotropy_type,star_number):


    file_name = f'Cij_cartesian_stars_{star_index}_{profile_type}_beta_{anisotropy_type}.txt'
    covariance_star_path = file_path = os.path.join(covariance_directory, f'star{star_number}')
    file_path = os.path.join(covariance_star_path, file_name)
    
    matrix = np.loadtxt(file_path)
    return matrix

In [19]:
#Load the mock data
mock_data_file = f'Mockdata_{space_type}_{star_number}_stars_{profile_type}_beta_{anisotropy_type}.txt'
mock_data_path = os.path.join(base_directory, mock_data_file)
mock_data = load_data(mock_data_path) #array [][] che contiene i dati

C:\Users\andre\AppData\Local\Temp\ipykernel_19800\3422306808.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(file_path, delim_whitespace=True, names=['x', 'y', 'z', 'vx', 'vy', 'vz'])


## Functions

# Likelyhood functions

In [20]:
# Calculate the sigma_vt^2 value for a given covariance matrix index
def calculate_sigma_vt2(matrix_index, covariance_matrices): #matrix index gli passero poi star_index
    
    covariance_matrix = covariance_matrices[matrix_index]
    
    sigma_vtheta_vtheta = covariance_matrix[1, 1]
    sigma_vphi_vphi = covariance_matrix[2, 2]
    
    sigma_vt2 = (sigma_vtheta_vtheta + sigma_vphi_vphi) / 2
    return sigma_vt2

#Create a 3x3 instrumental covariance matrix with specified sigma_v.
def create_S_instrumental(sigma_v):

    S_instrumental = np.zeros((3, 3))
    np.fill_diagonal(S_instrumental, sigma_v**2)
    return S_instrumental

def calculate_mu_v(mock_data):
    
    avg_vx = np.mean(mock_data[3])
    avg_vy = np.mean(mock_data[4])
    avg_vz = np.mean(mock_data[5])
    
    mu_v = np.array([avg_vx, avg_vy, avg_vz])

    return mu_v


def integrand(r, rho0, rs, gamma):
        return  r**2 * (rho0 / (((r / rs)**gamma) * (1 + r / rs)**(3 - gamma)))


def DM_profile_model(mock_data, params, star_index):
    # Unpack the parameters
    rho0, rs, gamma = params

    b = np.sqrt(mock_data[star_index][0]**2 + mock_data[star_index][1]**2 + mock_data[star_index][2]**2)

    integral, _ = integrate.quad(integrand, 0, b, args = (10**rho0, 10**rs, gamma))

    V_squared = (4 * np.pi * G / b) * abs(integral)
        
    V_r = np.sqrt(V_squared)
    
    return V_r



def p_v_given_r(star_index, mu_v, S_instrumental, covariance_matrix, ndim, mock_data, params):

    V_r = DM_profile_model(mock_data, params, star_index) #eh ma sta velocità è un numero non un vettore
    V_r_vett = [V_r, V_r, V_r]
    
    # Calculate relative velocity vector
    # v_relative = V_r - np.linalg.norm(mu_v)
    v_relative = V_r_vett - mu_v

    # Calculate S_total which is the sum of S_instrumental and covariance_matrix
    S_total = S_instrumental + covariance_matrix

    log_numerator = (-0.5 * np.dot(np.dot(np.transpose(v_relative), np.linalg.inv(S_total)), v_relative))
    log_denominator = 0.5*np.log10(((2 * np.pi)**ndim)*(np.linalg.det(S_total)))
    return log_numerator - log_denominator



#Likelyhood
def log_likelyhood(params):
    L_params = 0

    for star_index in range(star_number):
    
        covariance_matrix = covariance_matrices[star_index]
        position_cartesian = mock_data[star_index][:3]
        #v_cartesian = mock_data[star_index][3:]

        prob_given_r = p_v_given_r(star_index, mu_v, S_instrumental, covariance_matrix, ndim, mock_data, params)

        L_params += prob_given_r
    if not math.isnan(L_params):
        return -np.inf
    return L_params

## Main

In [21]:
# Print the first row of mock data
print("First row of mock data:")
print(mock_data[0])

First row of mock data:
[ 0.23216535 -0.02628695 -0.18520756  1.05255798 -6.33599615 -0.69663847]


In [22]:
# Load covariance matrices
#covariance_directory = os.path.join(base_directory, f'covariance_matrix\\star{star_number}')
covariance_matrices = []

for star_index in range(star_number):
    matrix = load_covariance_matrix(star_index, anisotropy_type, star_number)
    covariance_matrices.append(matrix) #array di matrici di covarianza

In [23]:
# Print the first covariance matrix
print(covariance_matrices[0])

[[67.496296  0.        0.      ]
 [ 0.       67.496296  0.      ]
 [ 0.        0.       67.496296]]


In [24]:
# Params definition
rho0 = 2.284
rs = -0.824
gamma = 0
#beta = 0


params = [rho0, rs, gamma]

sigma_v = 0  # Set this to 0, 1, or 5 as needed

mu_v = calculate_mu_v(mock_data)
S_instrumental = create_S_instrumental(sigma_v)

#p0 = np.random.uniform(low=[-3, -2, -3, -10], high=[5, 3, 3, 1], size=(nwalker, ndim)) #vettore con inizializzazione randomica dei parametri con beta
p0 = np.random.uniform(low=[-3, -2, -3], high=[5, 3, 3], size=(nwalker, ndim)) #vettore con inizializzazione randomica dei parametri

In [25]:
params

[2.284, -0.824, 0]

In [26]:
p0

array([[ 4.95000486e+00, -1.99845741e+00,  4.13054977e-01],
       [ 4.05945921e+00,  1.22765623e+00,  1.94838812e+00],
       [ 2.97730432e+00, -4.57722564e-01,  9.37616012e-01],
       [ 1.03881402e-01, -1.69759968e+00, -2.95274067e+00],
       [-1.81163590e+00,  3.73661665e-01,  2.21664251e+00],
       [ 9.60905078e-03,  1.52816525e+00, -1.95615914e+00],
       [ 4.73328214e+00,  2.57070313e+00, -2.08540460e+00],
       [ 1.11567439e+00,  8.35288545e-01,  4.37308649e-01],
       [ 4.68922249e+00,  6.44896635e-01,  9.70345831e-01],
       [ 3.54305997e+00, -1.35933694e+00,  1.12176401e+00],
       [ 3.53469495e+00,  2.57840587e-01,  7.20157467e-01],
       [ 4.84413236e+00, -1.34376011e+00,  2.43378196e+00],
       [ 7.73397513e-01,  2.01601076e+00, -2.97835832e-01],
       [ 6.09716321e-01,  1.92403416e+00, -2.16073713e-01],
       [ 3.62733316e+00, -8.34447141e-01, -2.78104251e+00],
       [ 6.09307442e-01,  4.62709655e-01,  1.43868436e+00],
       [ 4.41752157e+00,  5.47893051e-01

In [27]:
multiprocessing.set_start_method('spawn', force=True)


with multiprocessing.Pool() as pool:
    sampler = emcee.EnsembleSampler(nwalker, ndim, log_likelyhood, pool=pool)
    sampler.run_mcmc(p0, niter, progress = True)

In [ ]:
samples=sampler.get_chain(discard=10,thin=2,flat=True)
samples
# 2 // -0.8 // 0.2

array([[-0.58514413, -1.80627026, -1.63939476],
       [ 0.16034372,  1.36280463, -2.9962559 ],
       [-2.55269144,  1.266652  , -2.51876003],
       ...,
       [-1.52040309, -1.21105891,  1.0369766 ],
       [-0.06284683,  0.45733106, -1.1989928 ],
       [ 3.33927858, -1.62341069, -0.89135119]])

In [ ]:
samples.shape

(12250, 3)

In [ ]:
samples1 = sampler.chain.reshape((-1, ndim))

om_mc, H0_mc = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]), zip(*np.percentile(samples1, [16, 50, 84], axis=0)))

#Plotting the traces
thin, discard = 2, 10
fig, axes = plt.subplots(ndim, figsize=(18,9), sharex=True)

#plt.figure(figsize=(18,9))

for i in range(ndim):
    ax = axes[i]
    #plt.subplot(2,ndim//2,i+1) #this is for making two rows.
    ax.plot(sampler.get_chain()[thin:,discard:,i], color='black',alpha=0.1, lw=0.15)
    ax.set_xlim(0, len(sampler.get_chain()))
    ax.set_ylabel(params[i])

axes[-1].set_xlabel("step number");
plt.tight_layout ()

fig=corner.corner(samples,labels=params,show_titles=True,color='green',Truths=[om_mc,H0_mc],smooth=2.,Levels=[0.68,0.95],plot_density=True,plot_datapoints=False)

fig.savefig("corner_plot1.pdf")

ValueError: too many values to unpack (expected 2)